**Longitudal analysis**

In [1]:
import os


In [2]:
import pandas as pd

In [27]:
import qiime2
from qiime2.plugins import longitudinal, feature_table
from qiime2.plugins.longitudinal.pipelines import feature_volatility, first_differences
from qiime2 import Artifact, Metadata

import biom

ImportError: cannot import name 'first_differences'

In [4]:
sm_dir = os.path.abspath('../../../../..')
q2_out = os.path.join(sm_dir, 'data/tmp/microbiome/quant/qiime2/silva')
table_fn = os.path.join(q2_out, "table.qza")
sample_info = os.path.join(q2_out, "q2_sample_info.tsv")
feature_info = os.path.join(q2_out, "taxonomy.qza")

In [5]:
X = Artifact.load(table_fn)
S = Metadata.load(sample_info)
M = Artifact.load(feature_info)
df = S.to_dataframe()

In [6]:
# model
models = {'main': {'subset': 'Subset::sample', 'state':'Dive_Num', 'subject': 'Diver'}}


In [7]:
df_f = df.loc[df.Subset=="sample",:]
S_f = Metadata(df_f)
X_f = feature_table.methods.filter_samples(X, metadata=S_f).filtered_table

In [24]:
fv = feature_volatility(table=X_f, metadata=S_f, state_column=models['main']['state'], individual_id_column=models['main']['subject'], 
                        cv=10, n_jobs=24, parameter_tuning=True, feature_count=20, importance_threshold="q3", estimator="RandomForestRegressor", n_estimators=1000)

/mnt/scratch/flatberg/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [25]:
fv.volatility_plot

<visualization: Visualization uuid: 74b8ee74-78ab-42d7-b5a7-977fd7892800>

In [37]:
d1 = longitudinal.methods.first_differences(metadata=S_f, state_column=models['main']['state'], individual_id_column=models['main']['subject'],
                                            metric="BMI")


In [51]:
dist = Artifact.load(os.path.join(q2_out, "diversity/metrics/bray_curtis_distance_matrix.qza"))

In [53]:
d2 = longitudinal.methods.first_distances(dist, S_f, state_column=models['main']['state'], individual_id_column=models['main']['subject'], baseline=0)

In [56]:
df_2 = d2.first_distances.view(pd.Series)

Call signature:
longitudinal.methods.first_differences(
    metadata:Metadata,
    state_column:Str,
    individual_id_column:Str,
    metric:Str,
    replicate_handling:Str % Choices('error', 'random', 'drop')='error',
    baseline:Float=None,
    table:FeatureTable[RelativeFrequency]=None,
) -> (SampleData[FirstDifferences],)
Type:           Method
String form:    <method qiime2.plugins.longitudinal.methods.first_differences>
File:           /mnt/scratch/flatberg/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/qiime2/sdk/action.py
Docstring:      QIIME 2 Method
Call docstring:
Compute first differences or difference from baseline between sequential
states

Calculates first differences in "metric" between sequential states for
samples collected from individual subjects sampled repeatedly at two or
more states. First differences can be performed on a metadata column
(including artifacts that can be input as metadata) or a feature in a
feature table. Outputs a data series of 

In [57]:
longitudinal.visualizers.linear_mixed_effects?

Call signature:
longitudinal.visualizers.linear_mixed_effects(
    metadata:Metadata,
    state_column:Str,
    individual_id_column:Str,
    metric:Str=None,
    group_columns:Str=None,
    random_effects:Str=None,
    table:FeatureTable[RelativeFrequency]=None,
    palette:Str % Choices('Set1', 'Set2', 'Set3', 'Pastel1', 'Pastel2', 'Paired', 'Accent', 'Dark2', 'tab10', 'tab20', 'tab20b', 'tab20c', 'viridis', 'plasma', 'inferno', 'magma', 'terrain', 'rainbow', 'cividis')='Set1',
    lowess:Bool=False,
    ci:Float % Range(0, 100)=95,
    formula:Str=None,
) -> (Visualization,)
Type:           Visualizer
String form:    <visualizer qiime2.plugins.longitudinal.visualizers.linear_mixed_effects>
File:           /mnt/scratch/flatberg/miniconda3/envs/qiime2-2019.10/lib/python3.6/site-packages/qiime2/sdk/action.py
Docstring:      QIIME 2 Visualizer
Call docstring:
Linear mixed effects modeling

Linear mixed effects models evaluate the contribution of exogenous
covariates "group_columns" and 